In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import streamlit as st
import joblib
from imblearn.over_sampling import SMOTE

# Step 1: Load Dataset
def load_dataset(file_path):
    data = pd.read_csv('Natural_Disaster_Prediction_v2.csv')
    # Encode disaster type for classification
    data['Disaster Type'] = data['Disaster Type'].astype('category').cat.codes
    return data

# Step 2: Train Models
def train_models(data):
    # Features and labels for disaster occurrence
    X = data[['Rainfall (mm)', 'Humidity (%)', 'Temperature (°C)', 'Seismic Activity (Magnitude)', 'Wind Speed (km/h)']]
    y_occurrence = (data['Disaster Type'] > 0).astype(int)  # 1 if a disaster occurred, 0 otherwise
    y_type = data['Disaster Type']

    # Handle class imbalance using SMOTE
    smote = SMOTE(random_state=42)
    X_resampled, y_occurrence_resampled = smote.fit_resample(X, y_occurrence)

    # Train model for disaster occurrence
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_occurrence_resampled, test_size=0.3, random_state=42)
    occurrence_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=15)
    occurrence_model.fit(X_train, y_train)
    occurrence_accuracy = accuracy_score(y_test, occurrence_model.predict(X_test))
    print(f"Disaster Occurrence Model Accuracy: {occurrence_accuracy:.2f}")

    # Train model for disaster type prediction (only for rows where disaster occurred)
    X_type = X[y_occurrence == 1]
    y_type_filtered = y_type[y_occurrence == 1]
    X_type_train, X_type_test, y_type_train, y_type_test = train_test_split(X_type, y_type_filtered, test_size=0.3, random_state=42)
    type_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=15)
    type_model.fit(X_type_train, y_type_train)
    type_accuracy = accuracy_score(y_type_test, type_model.predict(X_type_test))
    print(f"Disaster Type Model Accuracy: {type_accuracy:.2f}")

    # Save models
    joblib.dump(occurrence_model, 'disaster_occurrence_model.pkl')
    joblib.dump(type_model, 'disaster_type_model.pkl')

# Step 3: Deploy with Streamlit
def deploy_with_streamlit():
    st.title("Natural Disaster Prediction")
    st.sidebar.header("Input Conditions")
    st.sidebar.write("Adjust the inputs below to predict natural disasters.")

    # Input fields
    temperature = st.sidebar.slider("Temperature (°C)", 15.0, 45.0, 25.0)
    rainfall = st.sidebar.slider("Rainfall (mm)", 0.0, 300.0, 50.0)
    seismic_activity = st.sidebar.slider("Seismic Activity (Magnitude)", 0.0, 10.0, 0.0)
    wind_speed = st.sidebar.slider("Wind Speed (km/h)", 0.0, 200.0, 10.0)
    humidity = st.sidebar.slider("Humidity (%)", 20.0, 100.0, 50.0)

    # Prepare input data
    input_data = pd.DataFrame({
        'Rainfall (mm)': [rainfall],
        'Humidity (%)': [humidity],
        'Temperature (°C)': [temperature],
        'Seismic Activity (Magnitude)': [seismic_activity],
        'Wind Speed (km/h)': [wind_speed]
    })

    # Load models
    occurrence_model = joblib.load('disaster_occurrence_model.pkl')
    type_model = joblib.load('disaster_type_model.pkl')

    # Predict disaster occurrence
    occurrence_prediction = occurrence_model.predict(input_data)[0]
    if occurrence_prediction == 1:
        # Predict disaster type
        type_prediction = type_model.predict(input_data)[0]
        disaster_types = {0: 'None', 1: 'Flood', 2: 'Drought', 3: 'Cyclone/Storm', 4: 'Earthquake/Tsunami', 5: 'Wildfire', 6: 'Hurricane'}
        st.warning(f"Disaster Predicted: {disaster_types[type_prediction]}")
    else:
        st.success("No disaster predicted.")

# Main Script
if __name__ == "__main__":
    # Load the dataset
    dataset_path = "Natural_Disaster_Prediction_v2.csv"
    data = load_dataset(dataset_path)

    # Train models
    train_models(data)

    # Deploy application
    deploy_with_streamlit()


Disaster Occurrence Model Accuracy: 0.99


2025-01-18 22:44:24.134 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Disaster Type Model Accuracy: 1.00


2025-01-18 22:44:24.990 
  command:

    streamlit run c:\Users\divya\OneDrive\Desktop\AI projects\ayush_ND\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-18 22:44:24.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 22:44:24.995 Thread 'MainThread': missing ScriptRunContext! This war